In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
def get_jobs(keyword, num_jobs, verbose, path, sleep_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    url = 'https://www.glassdoor.co.uk/Job/' + keyword + '-jobs-SRCH_KO0,14.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=' + keyword + '&typedLocation=&context=Jobs&dropdown=0'
#     url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(sleep_time)

        
        #Going through each job in this page
        job_buttons = driver.find_elements(By.CLASS_NAME, "react-job-listing")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            #Test for the "Sign Up" prompt and get rid of it.
            try:
                driver.find_element(By.CLASS_NAME, "selected").click()
            except ElementClickInterceptedException:
                pass

            time.sleep(.1)

            try:
                driver.find_element(By.CLASS_NAME, "modal_closeIcon-svg").click()  #clicking to the X.
            except NoSuchElementException:
                pass
            if len(jobs) >= num_jobs:
                break
            while not collected_successfully: 
                try:
                    company_name = driver.find_element(By.XPATH, '//div[@class="css-xuk5ye e1tk4kwz5"]').text
                    location = driver.find_element(By.XPATH, '//div[@class="css-56kyx5 e1tk4kwz1"]').text
                    job_title = driver.find_element(By.XPATH, '//div[@class="css-1j389vi e1tk4kwz2"]').text
                    job_description = driver.find_element(By.XPATH, '//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                    print("success")
                except:
                    print("no success")
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element(By.XPATH, '//span[@class="css-1hbqxax e1wijj240"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element(By.XPATH, '//span[@class="css-1m5m32b e1tk4kwz4"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element(By.XPATH, '//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element(By.XPATH, '//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element(By.XPATH, '//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element(By.XPATH, '//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element(By.XPATH, '//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element(By.XPATH, '//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element(By.XPATH, '//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element(By.XPATH, '//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element(By.XPATH, '//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element(By.XPATH, '//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [3]:
path = "/Users/gurpa/Documents/data-science/projects/salary_predictor/chromedriver"
df = get_jobs('data scientist', 15, True, path, 15)

C:\Users\gurpa\AppData\Local\Temp\ipykernel_40116\660748801.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path, options=options)


Progress: 0/15
success
Job Title: Data Scientist
Salary Estimate: Employer Provided Salary:£50 - £60 Per Hour
Job Description: CK Group is recruiting for a Data Scientist on a 12 month contract basis to join a leading multinational biopharmaceutical company. The role has been deemed to be outside IR35 and working options of Ltd company, umbrella and PAYE via CK Group are all available.

RESPONSIBILITIES:
As Data Scientist you will:
Lead, design, develop, execute and deliver the optimal high quality data integration solutions needed for clinical trial and biomarker data exploration to support successful strategic decisio
Rating: 5.0
Company Name: CK Group
5.0
Location: Welwyn Garden City, England
Headquarters: -1
Size: -1
Founded: -1
Type of Ownership: -1
Industry: -1
Sector: -1
Revenue: -1
Competitors: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 1/15
success
Job Title: Data Scientist
Salary Estimate: Employer Provided Salary:£40K - £65K
Job Description: YouLend is

success
Job Title: Data Scientist
Salary Estimate: Employer Provided Salary:£40K - £55K
Job Description: Salary will be completely negotiable and dependent on relevant background and experience and will include significant company benefits.
The client, a growing technology business with lots of opportunity to develop a career and based in central Manchester are seeking a Data Scientist to join their team at this exciting time of growth
This is a great opportunity to work on some exciting short and long term projects of a varied and interesting nature, in a business that is continuing to grow and ev
Rating: -1
Company Name: Marstep
Location: Manchester, England
Headquarters: -1
Size: -1
Founded: -1
Type of Ownership: -1
Industry: -1
Sector: -1
Revenue: -1
Competitors: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 11/15
success
Job Title: Apprentice (Degree Level) Data Scientist
Salary Estimate: Employer Provided Salary:£34K
Job Description: Do you have a passion for 

In [4]:
display(df)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,Employer Provided Salary:£50 - £60 Per Hour,CK Group is recruiting for a Data Scientist on...,5.0,CK Group\n5.0,"Welwyn Garden City, England",-1,-1,-1,-1,-1,-1,-1,-1
1,Data Scientist,Employer Provided Salary:£40K - £65K,YouLend is the preferred embedded financing pl...,4.4,YouLend\n4.4,"London, England",-1,-1,-1,-1,-1,-1,-1,-1
2,NLP Data Scientist,Employer Provided Salary:£65K - £75K,NLP Data Scientist (Python SQL) *Hybrid WfH*. ...,3.9,client server\n3.9,"London, England",-1,-1,-1,-1,-1,-1,-1,-1
3,Senior Scientist in Data Assimilation,Employer Provided Salary:£52K,Our reward package\nWe're looking for an excep...,4.1,Met Office\n4.1,"Exeter, England",-1,-1,-1,-1,-1,-1,-1,-1
4,Data Scientist/DE,Employer Provided Salary:£40K - £50K,Fuelled by a fundamental belief that having ac...,4.4,CarbonLaces\n4.4,"London, England",-1,-1,-1,-1,-1,-1,-1,-1
5,Foundation Scientist in Data Assimilation,Employer Provided Salary:£34K,Our reward package\nWe're looking for an excep...,4.1,Met Office\n4.1,"Exeter, England",-1,-1,-1,-1,-1,-1,-1,-1
6,Scientist in Data Assimilation,Employer Provided Salary:£43K,Our reward package\nWe're looking for an excep...,4.1,Met Office\n4.1,"Exeter, England",-1,-1,-1,-1,-1,-1,-1,-1
7,Junior Data Scientist,Employer Provided Salary:£23K - £30K,This role pays up to £30k (dependent on experi...,3.0,Inspiring Interns\n3.0,"London, England",-1,-1,-1,-1,-1,-1,-1,-1
8,Senior Data Scientist,Employer Provided Salary:£60K - £70K,The Company\nA well-funded and exciting start-...,-1,Metrica Recruitment,"London, England",-1,-1,-1,-1,-1,-1,-1,-1
9,Data Scientist - R& Python,Employer Provided Salary:£35K - £40K,The Company\nA well-established location strat...,-1,Metrica Recruitment,"London, England",-1,-1,-1,-1,-1,-1,-1,-1
